In [ ]:
%matplotlib inline

import rasterio
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt

out = r'O:\VDCNS\protocolo\pro\20150119l8oli202_32\miracle.img'
b5 = r'O:\VDCNS\protocolo\pro\20150119l8oli202_32\20150119l8oli202_32_water_rec_b5.img'

with rasterio.open(b5) as src:
    B5 = src.read()
    print('B5', B5.shape, type(B5))

#A = np.random.randint(1, 5, size=(100, 100)).astype(np.double)
#y, x = np.ogrid[-50:50, -50:50]
mask = (B5 > 1500)
print('Mask', mask.shape, type(mask))
B5[~mask] = 0

# rank 2 structure with full connectivity
struct = ndimage.generate_binary_structure(3, 3)
print('Struct', struct.shape, type(struct))
erode = ndimage.binary_erosion(mask, struct)
edges = mask ^ erode

nonzero_idx = np.vstack(np.where(mask)).T
nonzero_vals = B5[mask]

tree = cKDTree(nonzero_idx)

edge_idx = np.vstack(np.where(edges)).T
neighbours = tree.query_ball_point(edge_idx, r=1, p=np.inf)

new_vals = np.hstack(np.mean(nonzero_vals[n]) for n in neighbours)

B_new = B5.astype(np.double, copy=True)
B_new[edges] = new_vals

profile = src.meta
profile.update(dtype=rasterio.int16)

with rasterio.open(out, 'w', **profile) as dst:
    dst.write(B_new.astype(rasterio.int16))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 4), sharex=True, sharey=True)
#norm = plt.Normalize(0, B5.max())
ax[0].imshow(edges)
ax[0].set_title('Original', fontsize='x-large')
ax[1].imshow(edges)
ax[1].set_title('Edges', fontsize='x-large')
ax[2].imshow(edges)
ax[2].set_title('Averaged', fontsize='x-large')
for aa in ax:
    aa.set_axis_off()
ax[0].set_xlim(20, 50)
ax[0].set_ylim(50, 80)
fig.tight_layout()
plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import cKDTree

A = np.random.randint(1, 5, size=(100, 100)).astype(np.double)
y, x = np.ogrid[-50:50, -50:50]
mask = x**2 + y**2 <= 30**2
A[~mask] = 0

# rank 2 structure with full connectivity
struct = ndimage.generate_binary_structure(2, 2)
erode = ndimage.binary_erosion(mask, struct)
edges = mask ^ erode

# the indices of the non-zero locations and their corresponding values
nonzero_idx = np.vstack(np.where(mask)).T
nonzero_vals = A[mask]

# build a k-D tree
tree = cKDTree(nonzero_idx)

# use it to find the indices of all non-zero values that are at most 1 pixel
# away from each edge pixel
edge_idx = np.vstack(np.where(edges)).T
neighbours = tree.query_ball_point(edge_idx, r=1, p=np.inf)

# take the average value for each set of neighbours
new_vals = np.hstack(np.mean(nonzero_vals[n]) for n in neighbours)

# use these to replace the values of the edge pixels
A_new = A.astype(np.double, copy=True)
A_new[edges] = new_vals

fig, ax = plt.subplots(1, 3, figsize=(10, 4), sharex=True, sharey=True)
norm = plt.Normalize(0, A.max())
ax[0].imshow(A, norm=norm)
ax[0].set_title('Original', fontsize='x-large')
ax[1].imshow(edges)
ax[1].set_title('Edges', fontsize='x-large')
ax[2].imshow(A_new, norm=norm)
ax[2].set_title('Averaged', fontsize='x-large')
for aa in ax:
    aa.set_axis_off()
ax[0].set_xlim(20, 50)
ax[0].set_ylim(50, 80)
fig.tight_layout()
plt.show()

# Get the  bordes of the raster

In [ ]:
%matplotlib inline

import rasterio, scipy
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import cKDTree
from scipy.spatial import KDTree


out = r'O:\VDCNS\protocolo\pro\19840910l5tm202_32\m3class_reclass_binary2222.img'
out2 = r'O:\VDCNS\protocolo\pro\19840910l5tm202_32\m3class_reclass_ternary2222.img'
out3 = r'O:\VDCNS\protocolo\pro\19840910l5tm202_32\solution2222.img'
b5 = r'O:\VDCNS\protocolo\pro\19840910l5tm202_32\19840910l5tm202_32_water_reclass_3.img'

with rasterio.open(b5) as src:
    B5 = src.read()
    
    B4 = B5.reshape(453, 484)
    #B3 = B4[B4 != 1] = 0 #vamos a probar a hacer todo desde este raster, para ello reclasifico primero a 2 clases
    B3 = np.where(B4 != 1, 0, B4)
    print(B3.shape)
    #B4[(B4 <= 1450) & (B4 > 0)] = 1
    #B4[B4 == np.nan] = 1000
    mask = ((B3 != 1))
    mask2 = ((B4 != 1))
    #B4[~mask] = 2
    
    struct = ndimage.generate_binary_structure(2, 2)
    erode = ndimage.binary_erosion(B3, struct)
    edges = mask ^ erode
    e = edges.reshape(1, 453, 484)
    print(edges.shape)

    
    #Probamos a hacer la ternaria
    struct2 = ndimage.generate_binary_structure(2, 2)
    erode2 = ndimage.binary_erosion(B4, struct2)
    edges2 = mask2 ^ erode2
    e2 = edges2.reshape(1, 453, 484)
    
    edges3 = np.where((e2 == 1) & (e == 0), 1, 0)
    e3 = edges3.reshape(1, 453, 484)
    
    profile = src.meta
    profile.update(dtype=rasterio.int32)


#plt.figure(1)   
plt.figure(figsize=(15,10))
plt.imshow(edges2)
'''h = plt.imshow(edges, cmap='viridis')
plt.figure(2)   
plt.figure( figsize=(20,10))
f = plt.imshow(edges2, cmap='viridis')
plt.figure(2)   
plt.figure( figsize=(20,10))
g = plt.imshow(edges3, cmap='viridis')'''
#plt.plot()
    
with rasterio.open(out2, 'w', **profile) as dst:
    dst.write(e2.astype(rasterio.int32))

with rasterio.open(out, 'w', **profile) as dst:
    dst.write(e.astype(rasterio.int32))
    
with rasterio.open(out3, 'w', **profile) as dst:
    dst.write(e3.astype(rasterio.int32))

In [ ]:
##CON ESTE JUNTO CON EL ARRIBA LO OBTENGO COMBINANDO LOS 2!!


out = r'O:\VDCNS\protocolo\pro\pr\edges\water_reclass_equal_1.img'
b5 = r'O:\VDCNS\protocolo\pro\20070310l7etm202_32\20070310l7etm202_32_water_reclass.img'

with rasterio.open(b5) as src:
    B5 = src.read()
    
    B4 = B5.reshape(453, 484)
    #B4[B4 == 1] = 0
    print(B4.shape)
    #B4[(B4 <= 1450) & (B4 > 0)] = 1
    #B4[B4 == np.nan] = 1000
    mask = ((B4 == 1)) 
    #B4[~mask] = 2
    
    struct = ndimage.generate_binary_structure(2, 2)
    erode = ndimage.binary_erosion(B4, struct)
    edges = mask ^ erode
    e = edges.reshape(1, 453, 484)
    print(edges.shape)
    profile = src.meta
    profile.update(dtype=rasterio.int32)

    
    
#plt.figure(figsize=(20,10))
#plt.imshow(edges, cmap='viridis')
    
with rasterio.open(out, 'w', **profile) as dst:
    dst.write(e.astype(rasterio.int32))

In [ ]:
import rasterio, scipy
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import pyplot as plt

out = r'O:\VDCNS\protocolo\pro\pr\edges_100.img'
b5 = r'O:\VDCNS\protocolo\pro\pr\3classes_gapfill.img'

with rasterio.open(b5) as src:
    B5 = src.read()

    B4 = B5.reshape(453, 484)
    mask2 = (B4 == 2)
    mask1 = (B4 == 1)

struct = ndimage.generate_binary_structure(2, 2)
erode = ndimage.binary_erosion(B4, struct)
edges2 = mask2 ^ erode
egdes1 = mask1 ^ erode

suma = edges2 + egdes1

e = suma.reshape(1, 453, 484)

profile = src.meta
profile.update(dtype=rasterio.int32)

with rasterio.open(out, 'w', **profile) as dst:
    dst.write(e.astype(rasterio.int32))

In [ ]:
import numpy as np
from scipy import ndimage as nd

def fill(data, invalid=None):
    """
    Replace the value of invalid 'data' cells (indicated by 'invalid') 
    by the value of the nearest valid data cell

    Input:
        data:    numpy array of any dimension
        invalid: a binary array of same shape as 'data'. True cells set where data
                 value should be replaced.
                 If None (default), use: invalid  = np.isnan(data)

    Output: 
        Return a filled array. 
    """
    #import numpy as np
    #import scipy.ndimage as nd

    if invalid is None: invalid = np.isnan(data)

    ind = nd.distance_transform_edt(invalid, return_distances=False, return_indices=True)
    return data[tuple(ind)]

In [ ]:
fill(B5, 0)

In [ ]:
%pylab inline
import rasterio
import numpy as np
import matplotlib.pyplot as plt

out3 = r'O:\VDCNS\protocolo\pro\19840910l5tm202_32\solution__.img'
b5 = r'O:\VDCNS\protocolo\pro\pr\3classes_gapfill.img'


with rasterio.open(b5) as src:
    B5 = src.read()
    B4 = B5.reshape(453, 484)

# make empty copy of array containing class values
borders = np.zeros_like(B4, dtype=int)

# get the size of the input array in both dimensions
x_size, y_size = 453,484
print(x_size, y_size)

# iterate through each value
for i in range(x_size):
    for j in range(y_size):
        # select those which are in class 1
        if B4[i,j] == 1 :
            # get slice of all adjacent pixels
            slice = B4[i-1:i+1,j-1:j+1]
            # check if any belong to class 2
            if np.any(slice == 2):
                # update borders array
                borders[i,j] = 1
                
plt.figure(figsize=(20,10))
plt.imshow(borders, cmap='viridis')
borders.reshape(1, 453, 484)
profile = src.meta
profile.update(dtype=rasterio.int32)
with rasterio.open(out, 'w', **profile) as dst:
    dst.write(borders.astype(rasterio.int32))

In [ ]:
!gdalinfo -stats C:\bust_temp\AHS_110521_P21ID_PT1234_w_Q_R_E_masked

In [ ]:
%pylab inline

import rasterio
import matplotlib.pyplot as plt
c = 0
ahs = r'C:\bust_temp\AHS_110521_P21ID_PT1234_w_Q_R_E_masked'
prof = []
with rasterio.open(ahs) as src:
    B = src.read()
    for n, e in enumerate(B):
        if B[n].mean() != 0:
            c += 1
            #print(n, c)
            prof.append((B[n][500,936], B[n][515,966], B[n][383,1025], B[n][222,1307], B[n][766,1250]))
print(B.shape)
plt.figure(figsize=(20,7))           
plt.plot(prof)

In [ ]:
import fiona

shape = r'O:\VDCNS\protocolo\data\Extent_Square.shp'

def get_exten_shp(shape):
    
    X = []
    Y = []
    
    with fiona.open(shape) as src:
    
    #ft = next(src)
        for k, v in src.items():

            #print(k, v) AHORA ESTAMOS IMPRIMIENDO LAS COORDENADAS
            #if v['properties']['CLASS_NAME'] == 'CANCHALES':

            coord = (v['geometry']['coordinates'])
            for i in coord:
                for j in i:
                    X.append(j[0])
                    Y.append(j[1])

                lim_N = max(Y)
                lim_S = min(Y)
                lim_O = min(X)
                lim_E = max(X)

    print(lim_O, lim_E, lim_N, lim_S)
    
get_exten_shp(shape)

In [ ]:
!gdal_translate -of AAIGrid O:\VDCNS\protocolo\pro\20020616l7etm202_32\20020616l7etm202_32_edges.img \
    O:\VDCNS\protocolo\pro\20020616l7etm202_32\edges_points.xyz

In [ ]:
import gdal,ogr,os

# open raster file
raster = gdal.Open(r'O:\VDCNS\protocolo\pro\20020616l7etm202_32\20020616l7etm202_32_water_reclass.img')
band = raster.GetRasterBand(1)
a = band.ReadAsArray()
print(a.min(), a.max(), a.mean())
#create new shp file
newSHPfn = r'O:\VDCNS\protocolo\pro\20020616l7etm202_32\20020616l7etm202_32_plg.img'
shpDriver = ogr.GetDriverByName("ESRI Shapefile")
if os.path.exists(newSHPfn):
    shpDriver.DeleteDataSource(newSHPfn)
outDataSource = shpDriver.CreateDataSource(newSHPfn)
outLayer = outDataSource.CreateLayer(newSHPfn, geom_type=ogr.wkbLineString )

# polygonize
gdal.Polygonize(band, None, outLayer, 1) 

In [ ]:
import csv

outputCSV = r'O:\VDCNS\protocolo\pro\20020616l7etm202_32\points.csv'
out = csv.writer(open(outputCSV,"wb"), delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
#out.writerow(['X','Y','Value'])

raster = gdal.Open(r'O:\VDCNS\protocolo\pro\20020616l7etm202_32\20020616l7etm202_32_water_reclass.img')
band = raster.GetRasterBand(1)
a = band.ReadAsArray()

(height, width )=a.shape
for row in range(0,height):
    for col in range(0,width):
        out.writerow(a[row,col])

# DAR COTA A PUNTOS FROM RASTER EDGES

In [19]:
import rasterio
import numpy as np

edges = r'O:\VDCNS\protocolo\pro\19840910l5tm202_32\19840910l5tm202_32_edges.img'
dtm = r'O:\VDCNS\Curvas_MTN_Historico\dtm_mtn_318.img'
extract = r'O:\VDCNS\Curvas_MTN_Historico\dtm_mtn_318.img'

with rasterio.open(edges) as border:
    EDGES = border.read()
    
with rasterio.open(dtm) as mdt:
    DTM = mdt.read()
    
with rasterio.open(extract) as ex:
    EX = ex.read()

a = np.where(EX < 1, 0, EX)
b = a[a!=0]
#DTM_EDGES = np.where(EDGES == 1, DTM, 0)
print('max', b.max(), 'min', b.min(), 'mean', b.mean(), 'median', np.median(b))



max 361.915 min 272.311 mean 299.985 median 300.284
